In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sn
import plotly.express as px
import json
import datetime
#import geopandas as gpd
import fiona

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

In [85]:
DATA_PATH = 'C:\\Users\\comp\\Documents\\New York\\income_and_pop_by_tract\\ACSST5Y2018.S1903_2020-06-07T162212\\'

income = pd.read_csv(DATA_PATH + 'ACSST5Y2018.S1903_data_with_overlays_2020-06-07T162202.csv', skiprows = [1])
income.head(10)

,GEO_ID,NAME,S1903_C01_024M,S1903_C02_024E,S1903_C02_024M,S1903_C03_024E,S1903_C03_024M,S1903_C01_025E,S1903_C01_025M,S1903_C02_025E,S1903_C02_025M,S1903_C03_025E,S1903_C03_025M,S1903_C01_026E,S1903_C01_026M,S1903_C02_026E,S1903_C02_026M,S1903_C03_026E,S1903_C03_026M,S1903_C01_027E,S1903_C01_027M,S1903_C02_027E,S1903_C02_027M,S1903_C03_027E,S1903_C03_027M,S1903_C01_028E,S1903_C01_028M,S1903_C02_028E,S1903_C02_028M,S1903_C03_028E,S1903_C03_028M,S1903_C01_029E,S1903_C01_029M,S1903_C02_029E,S1903_C02_029M,S1903_C03_029E,S1903_C03_029M,S1903_C01_030E,S1903_C01_030M,S1903_C02_030E,S1903_C02_030M,S1903_C03_030E,S1903_C03_030M,S1903_C01_031E,S1903_C01_031M,S1903_C02_031E,S1903_C02_031M,S1903_C03_031E,S1903_C03_031M,S1903_C01_032E,...,S1903_C03_015M,S1903_C01_016E,S1903_C01_016M,S1903_C02_016E,S1903_C02_016M,S1903_C03_016E,S1903_C03_016M,S1903_C01_017E,S1903_C01_017M,S1903_C02_017M,S1903_C03_017E,S1903_C03_017M,S1903_C01_018E,S1903_C01_018M,S1903_C02_018E,S1903_C02_018M,S1903_C03_018E,S1903_C03_018M,S1903_C01_019E,S1903_C01_019M,S1903_C02_019E,S1903_C02_019M,S1903_C03_019E,S1903_C03_019M,S1903_C01_020E,S1903_C01_020M,S1903_C02_020E,S1903_C02_020M,S1903_C03_020E,S1903_C03_020M,S1903_C01_021E,S1903_C01_021M,S1903_C02_021E,S1903_C02_021M,S1903_C03_021E,S1903_C03_021M,S1903_C01_022E,S1903_C01_022M,S1903_C02_022E,S1903_C02_022M,S1903_C03_022E,S1903_C03_022M,S1903_C01_023E,S1903_C01_023M,S1903_C02_023E,S1903_C02_023M,S1903_C03_023E,S1903_C03_023M,S1903_C01_024E,S1903_C02_017E
0,1400000US36001000100,"Census Tract 1, Albany County, New York",61,47.9,10.9,49655,16085,108,51,24.7,10.2,35208,10362,68,51,15.5,10.9,-,**,39,31,8.9,7.3,-,**,13,15,3.0,3.5,-,**,0,12,0.0,7.1,-,**,69,37,15.8,7.6,12198,6433,191,71,43.6,13.9,32321,10981,142,...,12007,214,53,48.9,11.4,21000,18917,224,71,11.4,58750,17558,157,53,35.8,11.0,68807,9163,58,37,13.2,8.4,-,**,268,71,61.2,10.5,28750,14668,156,57,35.6,12.4,20600,7758,13,20,3.0,4.6,-,**,0,12,0.0,7.1,-,**,210,51.1
1,1400000US36001000200,"Census Tract 2, Albany County, New York",166,64.9,12.8,40391,25519,145,99,14.8,9.1,-,**,75,66,7.7,7.1,-,**,105,100,10.7,9.7,75842,30108,12,19,1.2,2.0,-,**,6,13,0.6,1.4,-,**,172,95,17.6,9.6,21375,17514,394,162,40.3,13.6,25104,23910,344,...,21364,487,183,49.8,14.3,22734,11202,491,154,14.3,48393,21615,247,105,25.3,10.0,76712,33690,65,79,6.6,8.1,-,**,575,182,58.8,13.5,24557,13490,380,177,38.9,15.2,21042,19394,156,81,16.0,8.1,-,**,42,43,4.3,4.4,-,**,635,50.2
2,1400000US36001000300,"Census Tract 3, Albany County, New York",175,53.3,9.5,46212,4223,347,153,28.9,11.3,36025,18601,98,65,8.2,5.6,-,**,54,57,4.5,4.6,-,**,45,43,3.7,3.7,-,**,17,24,1.4,2.0,-,**,232,125,19.3,9.8,23043,1833,598,210,49.8,13.8,38889,14019,278,...,7229,506,174,42.1,14.2,36600,17587,695,235,14.2,47438,14957,614,201,51.1,14.0,66500,31267,145,64,12.1,5.5,93110,27465,548,197,45.6,13.8,26397,18255,348,168,29.0,13.7,25735,18162,39,40,3.2,3.3,-,**,13,22,1.1,1.8,-,**,640,57.9
3,1400000US36001000401,"Census Tract 4.01, Albany County, New York",68,57.4,10.0,96719,16754,126,57,28.6,11.6,91429,77007,62,47,14.1,11.0,163333,20272,0,12,0.0,7.1,-,**,0,12,0.0,7.1,-,**,0,12,0.0,7.1,-,**,69,45,15.6,9.6,-,**,152,71,34.5,15.1,88712,24400,211,...,20372,142,34,32.2,7.1,143214,74892,299,64,7.1,97847,9122,337,95,76.4,16.1,132543,53322,93,55,21.1,12.3,144196,22286,81,69,18.4,15.6,88523,16698,41,46,9.3,10.4,-,**,23,24,5.2,5.4,-,**,8,12,1.8,2.7,-,**,253,67.8
4,1400000US36001000403,"Census Tract 4.03, Albany County, New York",181,60.6,12.1,98423,17828,249,92,28.9,10.5,146354,37162,58,37,6.7,4.5,122167,30458,33,40,3.8,5.0,-,**,0,12,0.0,3.7,-,**,0,12,0.0,3.7,-,**,65,52,7.5,5.8,82417,18673,394,141,45.7,11.8,78194,42718,356,...,30040,328,97,38.0,9.6,93333,74480,535,143,9.6,125720,34399,622,135,72.1,10.3,126818,14226,174,78,20.2,9.7,120333,20216,157,80,18.2,9.5,63603,59438,88,61,10.2,7.4,-,**,84,95,9.7,10.0,-,**,66,88,7.6,9.5,-,**,523,62.0
5,1400000US36001000404,"Census Tract 4.04, Albany County, New York",17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,*

In [41]:
df_name = income['NAME'].str.split(',', expand=True)
county = df_name[1].str.split(' ', expand = True)[1]
df_name_2 = df_name[0].str.split(' ', expand=True)
df_name_2.head(500)
income2 = pd.concat([df_name_2[2],county,income], axis = 1)

In [42]:
income2.rename(columns = {1 : 'County', 2 : 'Tract'}, inplace = True)

In [43]:
income2.drop(columns = ['NAME'], inplace = True)

In [46]:
counties = ['Kings', 'Bronx', 'Richmond', 'Queens', 'New York']
income3 = income2[income2['County'].isin(counties)]
income3

,Tract,County,GEO_ID,S1903_C01_024M,S1903_C02_024E,S1903_C02_024M,S1903_C03_024E,S1903_C03_024M,S1903_C01_025E,S1903_C01_025M,S1903_C02_025E,S1903_C02_025M,S1903_C03_025E,S1903_C03_025M,S1903_C01_026E,S1903_C01_026M,S1903_C02_026E,S1903_C02_026M,S1903_C03_026E,S1903_C03_026M,S1903_C01_027E,S1903_C01_027M,S1903_C02_027E,S1903_C02_027M,S1903_C03_027E,S1903_C03_027M,S1903_C01_028E,S1903_C01_028M,S1903_C02_028E,S1903_C02_028M,S1903_C03_028E,S1903_C03_028M,S1903_C01_029E,S1903_C01_029M,S1903_C02_029E,S1903_C02_029M,S1903_C03_029E,S1903_C03_029M,S1903_C01_030E,S1903_C01_030M,S1903_C02_030E,S1903_C02_030M,S1903_C03_030E,S1903_C03_030M,S1903_C01_031E,S1903_C01_031M,S1903_C02_031E,S1903_C02_031M,S1903_C03_031E,S1903_C03_031M,...,S1903_C03_015M,S1903_C01_016E,S1903_C01_016M,S1903_C02_016E,S1903_C02_016M,S1903_C03_016E,S1903_C03_016M,S1903_C01_017E,S1903_C01_017M,S1903_C02_017M,S1903_C03_017E,S1903_C03_017M,S1903_C01_018E,S1903_C01_018M,S1903_C02_018E,S1903_C02_018M,S1903_C03_018E,S1903_C03_018M,S1903_C01_019E,S1903_C01_019M,S1903_C02_019E,S1903_C02_019M,S1903_C03_019E,S1903_C03_019M,S1903_C01_020E,S1903_C01_020M,S1903_C02_020E,S1903_C02_020M,S1903_C03_020E,S1903_C03_020M,S1903_C01_021E,S1903_C01_021M,S1903_C02_021E,S1903_C02_021M,S1903_C03_021E,S1903_C03_021M,S1903_C01_022E,S1903_C01_022M,S1903_C02_022E,S1903_C02_022M,S1903_C03_022E,S1903_C03_022M,S1903_C01_023E,S1903_C01_023M,S1903_C02_023E,S1903_C02_023M,S1903_C03_023E,S1903_C03_023M,S1903_C01_024E,S1903_C02_017E
88,1,Bronx,1400000US36005000100,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,...,**,0,17,-,**,-,**,0,17,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,17,-,**,-,**,0,-
89,2,Bronx,1400000US36005000200,128,36.1,12.2,41154,9336,203,85,21.2,8.2,65150,23218,213,100,22.2,10.4,123310,54265,92,74,9.6,7.4,-,**,12,20,1.3,2.1,-,**,92,55,9.6,5.8,-,**,144,75,15.0,7.2,22596,9617,408,110,42.6,10.7,66200,34194,...,16561,464,108,48.4,9.1,65735,49104,494,112,9.1,67000,37045,527,117,55.0,10.3,79358,34310,215,111,22.4,11.2,77694,15948,243,82,25.4,8.6,29390,4061,166,73,17.3,7.9,28081,4379,188,103,19.6,9.8,42404,38080,83,75,8.7,7.4,71250,48212,346,51.6
90,4,Bronx,1400000US36005000400,122,36.3,8.0,69479,23890,480,142,33.8,8.5,84500,16642,264,111,18.6,7.5,155455,83936,122,94,8.6,6.6,-,**,38,37,2.7,2.6,-,**,0,17,0.0,2.3,-,**,167,76,11.8,5.3,17866,11162,466,134,32.8,9.3,68125,8035,...,14249,614,139,43.3,8.2,72026,23543,805,145,8.2,101382,24692,683,150,48.1,9.0,102713,19004,209,101,14.7,6.8,103789,56089,508,122,35.8,8.3,64375,9451,289,117,20.4,8.3,68438,27632,228,118,16.1,7.7,115250,80900,116,84,8.2,5.6,-,**,515,56.7
91,16,Bronx,1400000US36005001600,131,45.0,7.6,34688,7841,333,108,21.9,6.8,41693,6492,363,116,23.8,7.2,34955,19761,89,55,5.8,3.6,-,**,29,30,1.9,2.0,-,**,24,37,1.6,2.5,-,**,252,95,16.5,5.8,11333,8655,746,127,49.0,7.5,33230,3358,...,9315,699,134,45.9,7.9,42113,11406,825,139,7.9,35817,10358,459,144,30.1,8.7,55134,31226,189,88,12.4,5.7,58295,15374,915,144,60.0,7.7,40410,10396,451,123,29.6,7.6,34821,9428,150,94,9.8,6.4,31196,9092,59,48,3.9,3.1,-,**,686,54.1
92,19,Bronx,1400000US36005001900,56,36.9,10.6,34688,26147,123,53,25.1,9.7,42292,12803,140,50,28.6,9.1,54375,49400,34,22,6.9,4.8,45972,7371,0,12,0.0,6.4,-,**,12,15,2.4,3.0,-,**,100,48,20.4,9.0,7250,5833,175,54,35.7,10.4,40677,12040,...,4720,318,62,64.9,9.0,44231,7331,172,51,9.0,38167,19208,232,64,47.3,10.4,51000,39083,152,54,31.0,9.7,75625,35607,214,55,43.7,9.9,28333,19688,133,47,27.1,9.1,29375,22197,44,30,9.0,6.0,44038,9924,33,27,6.7,5.5,42981,19026,181,35.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [94]:
income4 = income3.filter(['Tract', 'County', 'GEO_ID', 'S1903_C03_015E'])
#income4['med_family_income'] = income4['S1903_C03_015E'].astype(float)
income4['types'] = income4['S1903_C03_015E'].apply(lambda x : 'num' if x.isnumeric() else 'str')
income5 = income4[income4['types'] == 'num']
income5['med_family_income'] = income5['S1903_C03_015E'].astype(int)
income5['GEO_ID_11'] = income5['GEO_ID'].apply(lambda x : x[-11:])

<ipython-input-94-d90d19359a5c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income5['med_family_income'] = income5['S1903_C03_015E'].astype(int)
<ipython-input-94-d90d19359a5c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  income5['GEO_ID_11'] = income5['GEO_ID'].apply(lambda x : x[-11:])


In [84]:
df_tracts

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,007,012702,36007012702,127.02,Census Tract 127.02,G5020,S,65411230,228281,+42.0350532,-075.9055509,"POLYGON ((-75.959172 42.008516, -75.959138 42...."
1,36,007,012800,36007012800,128,Census Tract 128,G5020,S,12342855,259439,+42.1305335,-075.9181575,"POLYGON ((-75.94766 42.137271, -75.944531 42.1..."
2,36,007,012900,36007012900,129,Census Tract 129,G5020,S,14480253,63649,+42.1522758,-075.9766029,"POLYGON ((-76.022294 42.156988, -76.019165 42...."
3,36,007,013000,36007013000,130,Census Tract 130,G5020,S,9934434,381729,+42.1236499,-076.0002197,"POLYGON ((-76.024168 42.125898, -76.02409 42.1..."
4,36,007,013202,36007013202,132.02,Census Tract 132.02,G5020,S,2451349,3681,+42.1238924,-076.0311921,"POLYGON ((-76.039964 42.132845, -76.0386219999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4913,36,027,640002,36027640002,6400.02,Census Tract 6400.02,G5020,S,2304853,13876,+41.5216636,-073.9534317,"POLYGON ((-73.96802199999999 41.516966, -73.96..."
4914,36,027,130004,36027130004,1300.04,Census Tract 1300.04,G5020,S,24394743,363637,+41.7909753,-073.8382016,"POLYGON ((-73.86601399999999 41.814124, -73.86..."
4915,36,027,050104,36027050104,501.04,Census Tract 501.04,G5020,S,25446931,234551,+41.5929768,-073.7645636,"POLYGON ((-73.803893 41.58353, -73.80303499999..."
4916,36,119,014500,36119014500,145,Census Tract 145,G5020,S,2778952,14648,+41.2827060,-073.9087061,"POLYGON ((-73.92410599999999 41.273959, -73.92..."


In [96]:
df_merge = income5.merge(df_tracts, left_on = 'GEO_ID_11', right_on = 'GEOID', how = 'left')
df_merge

,Tract,County,GEO_ID,S1903_C03_015E,types,med_family_income,GEO_ID_11,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,2,Bronx,1400000US36005000200,66488,num,66488,36005000200,36,005,000200,36005000200,2,Census Tract 2,G5020,S,479463,852046,+40.8035344,-073.8606612,"POLYGON ((-73.865218 40.814529, -73.864255 40...."
1,4,Bronx,1400000US36005000400,90118,num,90118,36005000400,36,005,000400,36005000400,4,Census Tract 4,G5020,S,770689,690794,+40.8087867,-073.8514252,"POLYGON ((-73.85959699999999 40.815283, -73.85..."
2,16,Bronx,1400000US36005001600,40577,num,40577,36005001600,36,005,001600,36005001600,16,Census Tract 16,G5020,S,485079,0,+40.8188478,-073.8580764,"POLYGON ((-73.862464 40.819247, -73.861531 40...."
3,19,Bronx,1400000US36005001900,41447,num,41447,36005001900,36,005,001900,36005001900,19,Census Tract 19,G5020,S,1668013,1143135,+40.8009990,-073.9093729,"POLYGON ((-73.93180099999999 40.807876, -73.93..."
4,20,Bronx,1400000US36005002000,30690,num,30690,36005002000,36,005,002000,36005002000,20,Census Tract 20,G5020,S,395564,0,+40.8175801,-073.8660962,"POLYGON ((-73.87050099999999 40.820364, -73.86..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1811,291.04,Richmond,1400000US36085029104,93906,num,93906,36085029104,36,085,029104,36085029104,291.04,Census Tract 291.04,G5020,S,1541564,0,+40.6137234,-074.1635029,"POLYGON ((-74.17486599999999 40.613195, -74.17..."
1812,303.01,Richmond,1400000US36085030301,87847,num,87847,36085030301,36,085,030301,36085030301,303.01,Census Tract 303.01,G5020,S,719281,0,+40.6189197,-074.1561204,"POLYGON ((-74.168482 40.621562, -74.1681409999..."
1813,303.02,Richmond,1400000US36085030302,77422,num,77422,36085030302,36,085,030302,36085030302,303.02,Census Tract 303.02,G5020,S,908763,0,+40.6235288,-074.1587435,POLYGON ((-74.16814099999999 40.62207799999999...
1814,319.02,Richmond,1400000US36085031902,48071,num,48071,36085031902,36,085,031902,36085031902,319.02,Census Tract 319.02,G5020,S,690539,296692,+40.6379208,-074.1655701,"POLYGON ((-74.17128699999999 40.640518, -74.17..."


AttributeError: module 'string' has no attribute 'isnumeric'